In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import os
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn.metrics import precision_score
from sklearn.model_selection import cross_val_score
import time

import matplotlib
import matplotlib.pyplot as plt
from datetime import datetime

%matplotlib inline

# Description of the Problem

Kickstarter success rate. 


Training and Testing error. Training time. ROC curve



2. the training and testing error rates you obtained running the various learning algorithms on your problems. At the very least you should include graphs that show performance on both training and test data as a function of training size (note that this implies that you need to design a classification problem that has more than a trivial amount of data) and--for the algorithms that are iterative--training times/iterations. Both of these kinds of graphs are referred to as learning curves, BTW.

3. analyses of your results. Why did you get the results you did? Compare and contrast the different algorithms. What sort of changes might you make to each of those algorithms to improve performance? How fast were they in terms of wall clock time? Iterations? Would cross validation help (and if it would, why didn't you implement it?)? How much performance was due to the problems you chose? How about the values you chose for learning rates, stopping criteria, pruning methods, and so forth (and why doesn't your analysis show results for the different values you chose?)? Which algorithm performed best? How do you define best? Be creative and think of as many questions you can, and as many answers as you can.


-Why did you get the results you did? 

-Compare and contrast the different algorithms. 

-What sort of changes might you make to each of those algorithms to improve performance? 

-How fast were they in terms of wall clock time? Iterations? 

-Would cross validation help (and if it would, why didn't you implement it?)? How much performance was due to the 
problems you chose? 

-How about the values you chose for learning rates, stopping criteria, pruning methods, and so forth (and why doesn't your analysis show results for the different values you chose?)? 

-Which algorithm performed best? How do you define best? Be creative and think of as many questions you can, and as many answers as you can.


In [8]:
df = pd.read_csv('Kickstarter.csv')
df = df.drop(['ID', 'name'], axis=1)
df['deadline'] = pd.to_datetime(df['deadline'])
df['launched'] = pd.to_datetime(df['launched'])
df['duration'] = df['deadline'] - df['launched']
df['pledged_percentage'] = df['pledged']/df['goal']
actual_conditions = [
    (df['state'] == 'failed'), (df['state'] == 'canceled'), (df['state'] == 'successful')  ]

choices = [-1, 0, 1]
df['Result'] = np.select(actual_conditions, choices, default=0)
df['usd pledged'] = np.where(df['currency']=='USD', df['pledged'], df['usd pledged'])
df['deadline'] = [x.year for x in df['deadline']]
df['duration'] = (df['duration'] / np.timedelta64(1, 'D')).astype(int)
df = df.drop(['pledged', 'state','country', 'launched'], axis=1)

In [9]:
from sklearn import preprocessing

category = preprocessing.LabelEncoder()
category.fit(df['category'])
df['category'] = category.transform(df['category']) 

main_category = preprocessing.LabelEncoder()
main_category.fit(df['main_category'])
df['main_category'] = main_category.transform(df['main_category']) 

currency = preprocessing.LabelEncoder()
currency.fit(df['currency'])
df['currency'] = currency.transform(df['currency']) 

In [10]:
df = df.replace([np.inf, -np.inf], np.nan).dropna(axis=0, how='any')

In [11]:
df

,category,main_category,currency,deadline,goal,backers,usd pledged,duration,pledged_percentage,Result
0,108,12,5,2015,1000.0,0,0.000000,58,0.000000,-1
1,93,6,13,2017,30000.0,15,2421.000000,59,0.080700,-1
2,93,6,13,2013,45000.0,3,220.000000,45,0.004889,-1
3,90,10,13,2012,5000.0,1,1.000000,30,0.000200,-1
4,55,6,13,2015,19500.0,14,1283.000000,55,0.065795,0
5,123,7,13,2016,50000.0,224,52375.000000,35,1.047500,1
6,58,7,13,2014,1000.0,16,1205.000000,20,1.205000,1
7,41,7,13,2016,25000.0,40,453.000000,44,0.018120,-1
8,113,4,13,2014,125000.0,58,8233.000000,35,0.065864,0
9,39,6,13,2014,65000.0,43,6240.570000,30,0.096009,0


In [12]:
features = list(df.columns[0:-1])
X = df[features]
Y = df['Result']


#Shuffle = False b/c gotta train on earlier data and test on later data 
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.33, shuffle=False)

# 1. Decision Tree

In [13]:
DT = tree.DecisionTreeClassifier(criterion='gini', splitter='best', max_depth=None, min_samples_split=5, 
                                  min_samples_leaf=2, min_weight_fraction_leaf=0.0, max_features=None, 
                                  random_state=None, max_leaf_nodes=None, min_impurity_decrease=0.0, 
                                  min_impurity_split=None, class_weight=None, presort=False)
#DT = DT.fit(X_train, Y_train)
#Ypreds = DT.predict(X_test)

In [14]:
#The least populated class in y has only 8 members, which is too few. 
#The minimum number of members in any class cannot be less than n_splits=9.
scores = cross_val_score(DT, X_train, Y_train, cv = 8)

print scores
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

[ 0.84341502  0.83949407  0.84309881  0.84050593  0.84180237  0.84062737
  0.83694896  0.83954209]
Accuracy: 0.84 (+/- 0.00)


Pruning:

min_samples_split : int, float, optional (default=2)
The minimum number of samples required to split an internal node

min_samples_leaf : int, float, optional (default=1)
The minimum number of samples required to be at a leaf node:

# 2. Neural Networks (Multi-layer Perceptron (MLP) )

Pro:
*Capability to learn non-linear models.

Con: 
*MLP with hidden layers have a non-convex loss function where there exists more than one local minimum. Therefore different random weight initializations can lead to different validation accuracy.

*MLP requires tuning a number of hyperparameters such as the number of hidden neurons, layers, and iterations.

*MLP is sensitive to feature scaling.


## Features:
1. hidden_layer_sizes : tuple, length = n_layers - 2, default (100,)

The ith element represents the number of neurons in the ith hidden layer.
hidden_layer_sizes=(10, 2) : 10 hidden layer with 2 hidden units

2. activation : {‘identity’, ‘logistic’, ‘tanh’, ‘relu’}, default ‘relu’
Activation function for the hidden layer.

3. solver : {‘lbfgs’, ‘sgd’, ‘adam’}, default ‘adam’
The solver for weight optimization.

4. learning_rate : {‘constant’, ‘invscaling’, ‘adaptive’}, default ‘constant’
Learning rate schedule for weight updates.

In [80]:
from sklearn.neural_network import MLPClassifier

NN = MLPClassifier(hidden_layer_sizes=(20, 2), activation='relu', solver='adam', alpha=0.0001, batch_size='auto', 
 learning_rate='constant', learning_rate_init=0.001, power_t=0.5, max_iter=200, shuffle=True, 
 random_state=None, tol=0.0001, verbose=False, warm_start=False, momentum=0.9, nesterovs_momentum=True, 
 early_stopping=False, validation_fraction=0.1, beta_1=0.9, beta_2=0.999, epsilon=1e-08)


#NN = NN.fit(X_train, Y_train) 
#Ypreds = NN.predict(X_test)

In [81]:
scores = cross_val_score(NN, X_train, Y_train, cv = 10)

print scores
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

[ 0.62525692  0.62525692  0.62525692  0.62525692  0.62525692  0.62525692
  0.62528163  0.62526682  0.62526682  0.62526682]
Accuracy: 0.63 (+/- 0.00)


## Features:

1. n_estimators : integer, optional (default=10)
The number of trees in the forest.

2. criterion : string, optional (default=”gini”)
The function to measure the quality of a split. Supported criteria are “gini” for the Gini impurity and “entropy” for the information gain. 

3. max_features : int, float, string or None, optional (default=”auto”)
The number of features to consider when looking for the best split:

4. max_depth : integer or None, optional (default=None)
The maximum depth of the tree. If None, then nodes are expanded until all leaves are pure or until all leaves contain less than min_samples_split samples.

5. min_samples_split : int, float, optional (default=2)
The minimum number of samples required to split an internal node

6. min_samples_leaf : int, float, optional (default=1)
The minimum number of samples required to be at a leaf node

In [ ]:
from sklearn.ensemble import RandomForestClassifier

RF = RandomForestClassifier(n_estimators=10, criterion='gini', 
                            max_depth=None, min_samples_split=2, 
                            min_samples_leaf=1, min_weight_fraction_leaf=0.0, 
                            max_features='auto', max_leaf_nodes=None, 
                            min_impurity_decrease=0.0, min_impurity_split=None, 
                            bootstrap=True, oob_score=False, n_jobs=1, 
                            random_state=None, verbose=0, 
                            warm_start=False, class_weight=None)

#RF = RF.fit(X_train, Y_train) 
#Ypreds = RF.predict(X_test)

scores = cross_val_score(RF, X_train, Y_train, cv = 10)

print scores
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

[ 0.99059289  0.98940711  0.99102767  0.99059289  0.99189723  0.99102767
  0.98988102  0.99098743  0.99106649  0.98999921]
Accuracy: 0.99 (+/- 0.00)


# 4. SVM

-at least 2 kernel functions

Pros: 
* Effective in high dimensional spaces.
* Still effective in cases where number of dimensions is greater than the number of samples.
* Uses a subset of training points in the decision function (called support vectors), so it is also memory efficient.
* Versatile: different Kernel functions can be specified for the decision function. Common kernels are provided, but it is also possible to specify custom kernels.

Cons:

* If the number of features is much greater than the number of samples, avoid over-fitting in choosing Kernel functions and regularization term is crucial.
* SVMs do not directly provide probability estimates, these are calculated using an expensive five-fold cross-validation

## Features:

1. Kernel(Linear, RBF, poly, sigmoid, precomputed)

Linear kernel = Straight Line (hyperplane) as the decision boundary
* rarely used in practice

Radial Basis Function (RBF) = commonly used kernel in SVC
2 parameters:
* gamma
* C

2. Gamma:
*  'spread' of the kernel and therefore the decision region.
* low gamma -> the 'curve' of the decision boundary is very low and thus the decision region is very broad (underfitting)
* gamma = 10 (The decision boundary starts to be highly effected by individual data points (i.e. variance)).
* high gamma -> the 'curve' of the decision boundary is high, which creates islands of decision-boundaries around data points (overfitting)
* If gamma is ‘auto’ then 1/n_features will be used instead.
* gamma defines how much influence a single training example has. 
* The larger gamma is, the closer other examples must be to be affected.

3. C:
* trades off misclassification of training examples against simplicity of the decision surface
* A low C makes the decision surface smooth, while a high C aims at classifying all training examples correctly
* penalty for misclassifying a data point
* small C -> classifier is okay with misclassified data points (high bias, low variance)
* big C -> classifier is heavily penalized for misclassified data and therefore bends over backwards avoid any misclassified data points (low bias, high variance)

C > 10 is too slow

4. degree : int, optional (default=3)
* Degree of the polynomial kernel function (‘poly’). Ignored by all other kernels.

In [ ]:
#takes FOREVER
#linear 0.78
#rbf 0.76
#sigmoid 0.5

from sklearn.svm import SVC

kernels = ['linear', 'rbf', 'sigmoid']

for i in kernels:
    #for j, C in enumerate((0.01, 1, 10, 100)):
        #for k, D in enumerate((1, 10)):
    SVM = SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
              decision_function_shape='ovr', degree=3, gamma='auto', kernel=i,
              max_iter=-1, probability=False, random_state=None, shrinking=True,
              tol=0.001, verbose=False)
    #SVM.fit(X_train, Y_train) 
    scores = cross_val_score(SVM, X_train, Y_train, cv = 10)

    #print ("Kernel: %s | Gamma: %0.2f | C: %i" % (i, C, D))
    print ("Kernel: %s" % (i))
    print scores
    print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

# 5. KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

KNN = KNeighborsClassifier(n_neighbors=5, weights='uniform', algorithm='auto', 
                           leaf_size=30, p=2, metric='minkowski', metric_params=None, 
                           n_jobs=1)
#KNN.fit(X_train, Y_train)
scores = cross_val_score(KNN, X_train, Y_train, cv = 10)

print scores
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

## Features:

1. weights : str or callable, optional (default = ‘uniform’)
weight function used in prediction. Possible values:


* uniform : uniform weights. All points in each neighborhood are weighted equally.

* distance : weight points by the inverse of their distance. in this case, closer neighbors of a query point will have a greater influence than neighbors which are further away.

## II. Model Comparison

## 1. ROC Curves

In [ ]:
Adjusted R Square

ROC curve

auc Value

In [145]:
names = ["Decision Tree", "Neural Network", "Boosting", "SVM", "KNN"]

#implement ROC Curves
#

h = .02 

classifiers = [DT, NN, RF, SVM, KNN]